# Lab 5

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
D0 = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 5 - Culturing Hybirdoma Cells and Immodetection/Data/ELISA.csv',
                header=0)

In [3]:
D0.T

,0,1,2,3,4,5,6,7
Dilution,Mouse mAb alpha-HT 1,Mouse mAb alpha-HT 2,Mouse mAb alpha-HT sample 1,Mouse mAb alpha-HT sample 2,Mouse mAb alpha-OTI 1,Mouse mAb alpha-OTI 2,Rat mAb alpha-HT 1,Rat mAb alpha-HT 2
1,0.628,0.621,0.598,0.57,0.139,0.135,0.294,0.303
2,0.632,0.574,0.552,0.538,0.14,0.137,0.303,0.273
4,0.575,0.568,0.458,0.472,0.138,0.139,0.272,0.273
8,0.48,0.456,0.374,0.388,0.14,0.138,0.283,0.285
16,0.367,0.373,0.273,0.271,0.145,0.139,0.269,0.251
32,0.264,0.259,0.207,0.206,0.14,0.138,0.216,0.192
64,0.214,0.209,0.175,0.177,0.141,0.141,0.198,0.182
128,0.174,0.171,0.157,0.155,0.139,0.138,0.155,0.153
256,0.154,0.15,0.145,0.144,0.137,0.136,0.143,0.144


In [4]:
MH = pd.DataFrame(D0.iloc[0:2,:].mean(axis=0))
MH.columns = ['MH']

MS = pd.DataFrame(D0.iloc[2:4,:].mean(axis=0))
MS.columns = ['MS']

MO = pd.DataFrame(D0.iloc[4:6,:].mean(axis=0))
MO.columns = ['MO']

RH = pd.DataFrame(D0.iloc[6:8,:].mean(axis=0))
RH.columns = ['RH']

In [5]:
DT = pd.concat([MH, MS, MO, RH], sort=False, axis=1)
DT.columns = [r'Mouse mAb $\alpha$-HT (+ Control)', 
              r'Mouse mAb $\alpha$-HT (Sample)', 
              r'Mouse mAb $\alpha$-OTI (- Control)', 
              r'Rat mAb $\alpha$-HT (- Control)']
DT

,Mouse mAb $\alpha$-HT (+ Control),Mouse mAb $\alpha$-HT (Sample),Mouse mAb $\alpha$-OTI (- Control),Rat mAb $\alpha$-HT (- Control)
1,0.6245,0.5840,0.1370,0.2985
2,0.6030,0.5450,0.1385,0.2880
4,0.5715,0.4650,0.1385,0.2725
8,0.4680,0.3810,0.1390,0.2840
16,0.3700,0.2720,0.1420,0.2600
32,0.2615,0.2065,0.1390,0.2040
64,0.2115,0.1760,0.1410,0.1900
128,0.1725,0.1560,0.1385,0.1540
256,0.1520,0.1445,0.1365,0.1435
512,0.1435,0.1410,0.1355,0.1395


In [6]:
slope, intercept, r_value, p_value, std_err = stats.linregress([3,4], DT.iloc[3:5,1])
y = DT[r'Mouse mAb $\alpha$-HT (Sample)'].max()/2
x = (y - intercept)/slope

In [7]:
fig, ax = plt.subplots(figsize=(10, 4))
data = sns.lineplot(data=DT, sort=False, markers=['P', 'o', 'X', 'd'], dashes=False, lw=1.5)
titre = ax.plot([-1, x, x], [y, y, 0], linestyle='--', color='k', zorder=5, linewidth=1,
                label=r'Antibody titre: $^1/_{0:.3g}$ to $^1/_{{{1:.3g}}}$'.format(2**np.floor(x), 2**np.ceil(x)))

ax.legend()
ax.set_xlim(-0.5,)
ax.set_ylim(0.05,)
ax.set_xlabel('Antibody Dilution')
ax.set_ylabel(r'Average absorbance ($\bar{A}_{405}$)')

locs, labels = plt.xticks() 
x_tick_label1 = [round(2**float(n)) for n in range(len(locs))]
x_tick_label2 = [r'$^1/_{0}$'.format({i}) for i in x_tick_label1]
plt.xticks(locs, x_tick_label2, fontsize=12)

ax.grid()
plt.show()

<Figure size 720x288 with 1 Axes>